# Handling Categorical Data

## Encoding of Categorical Variables

In this section, we will present typical ways of dealing with categorical variables by encoding them, namely **ordinal encoding** and **one-hoe encoding**

So, without further due, let's start first by importing the required modules and loading the data set.

In [1]:
# 1. Standard imports
import pandas as pd

# Few tweaked Pandas options for friendlier output:

# Use 2 decimal places in output display
%precision %.3f
pd.set_option("display.precision", 3)

# Disable jedi autocompleter
%config Completer.use_jedi = False

In [2]:
df = pd.read_csv('data/adult-census.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# this time we are going to keep the categorical eduction variable; education and drop the numerical `education`
df = df.drop(columns='education-num')
target = df['class']
data = df.drop(columns='class')

### Identify Categorical Variables

As we saw in the previous section, a numerical variable is a quantity represented by a real or integer number. These variables can be naturally handled by machine learning algorithms that are typically composed of a sequence of arithmetic instructions such as additions and multiplications.

In contrast, categorical variables have discrete values, typically  represented by string labels taken from a finite list of possible choices. e.g:

In [4]:
data['native-country'].value_counts().sort_index()

 ?                               857
 Cambodia                         28
 Canada                          182
 China                           122
 Columbia                         85
 Cuba                            138
 Dominican-Republic              103
 Ecuador                          45
 El-Salvador                     155
 England                         127
 France                           38
 Germany                         206
 Greece                           49
 Guatemala                        88
 Haiti                            75
 Holand-Netherlands                1
 Honduras                         20
 Hong                             30
 Hungary                          19
 India                           151
 Iran                             59
 Ireland                          37
 Italy                           105
 Jamaica                         106
 Japan                            92
 Laos                             23
 Mexico                          951
 

There are different ways to select categorical data:

In [9]:
# 1.
data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
dtype: object

In [10]:
# 2.
data.select_dtypes(include='object').columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

### Select Features Based on their Data Type

scikit-learn comes with a helper function `make_column_selector`, which allows us to select columns based on their data type. Since this is a tutorial about sklearn, in the following we will illustrate how to use this helper.

In [17]:
from sklearn.compose import make_column_selector as selector

categorical_cols_selector = selector(dtype_include=object)
categorical_cols = categorical_cols_selector(data)
data_categorical = data[categorical_cols]
print(f"There are {len(categorical_cols)} categorical features in the dataset:")
categorical_cols

There are 8 categorical features in the dataset:


['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

## Strategies to Encode Categories

### Encoding Ordinal Categories

The most intuitive strategy is to encode each category with a different number using the `OrdinalEncoder`:

In [18]:
from sklearn.preprocessing import OrdinalEncoder

education_column = data_categorical[['education']]

encoder = OrdinalEncoder()
education_encoded = encoder.fit_transform(education_column)
education_encoded

array([[ 1.],
       [11.],
       [ 7.],
       ...,
       [11.],
       [11.],
       [11.]])

We see that each category in `education` has been replaced by a numeric value. We could check the mapping between the categories and the numerical values by checking the fitted attribute `categories_`.

In [19]:
encoder.categories_

[array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
        ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
        ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
        ' Some-college'], dtype=object)]

Now we can apply the encoder to all categorical features:

In [20]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

array([[ 4.,  1.,  4.,  7.,  3.,  2.,  1., 39.],
       [ 4., 11.,  2.,  5.,  0.,  4.,  1., 39.],
       [ 2.,  7.,  2., 11.,  0.,  4.,  1., 39.],
       [ 4., 15.,  2.,  7.,  0.,  2.,  1., 39.],
       [ 0., 15.,  4.,  0.,  3.,  4.,  0., 39.]])

In [21]:
encoder.categories_

[array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
        ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
       dtype=object),
 array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
        ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
        ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
        ' Some-college'], dtype=object),
 array([' Divorced', ' Married-AF-spouse', ' Married-civ-spouse',
        ' Married-spouse-absent', ' Never-married', ' Separated',
        ' Widowed'], dtype=object),
 array([' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair',
        ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners',
        ' Machine-op-inspct', ' Other-service', ' Priv-house-serv',
        ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support',
        ' Transport-moving'], dtype=object),
 array([' Husband', ' Not-in-family', ' Other-relative', ' Own-child',
        ' Unmarried'

In [22]:
len(encoder.categories_)

8

We see that the categories have been encoded for each feature independently. We also not that the number of features before and after the encoding is the same.

However, be careful when applying this encoding strategy: Using this integer representation leads downstream predictive models to assume that the values are ordered (0 < 1 < 2 < 3 ...)

By default, `OrdinalEncoder` uses an arbitrary and often meaningless strategy to map string category labels to integers. However, it accepts a `categories` constructor argument to pass categories in the expected ordering explicitly.

<div class="alert alert-block alert-warning">
As a rule of thumb: If a categorical variable does not carry any meaningful order information then avoid this type of encoding and consider using one-hot encoding instead.</div>